# Bottle Processing
Author: Andrew Reed
Date: 2021-01-12

### Purpose
This notebook combines the processed ```.btl``` files into a single Bottle File for use in generating the Discrete Sample Summary spreadsheet

### Data Sources/Software:

* **sbe_name_map**: This is a spreadsheet which maps the short names generated by the SeaBird SBE DataProcessing Software to the associated full names. The name mapping originates from SeaBird's SBE DataProcessing support documentation.

* **Alfresco**: The Alfresco CMS for OOI at alfresco.oceanobservatories.org is the source of the ctd hex, xmlcon, and psa files necessary for generating the bottle files needed to create the sample summary sheet.

* **SBEDataProcessing-Win32**: SeaBird vendor software for processing the raw ctd files and generating the .btl files.

### Import Libraries

In [1]:
import os
import sys
import re
import pandas as pd
import numpy as np

### Define functions

In [2]:
class Cast():

    def __init__(self, cast_number):
        self.cast_number = str(cast_number)

    def parse_header(self, header):
        """
        Parse the header of bottle (.btl) files to get critical information
        for the summary spreadsheet.

        Args:
            header - an object containing the header of the bottle file as a list of
                strings, split at the newline.
        Returns:
            self.hdr - a dictionary object containing the start_time, filename, latitude,
                longitude, and cruise id.
        """
        hdr = {}
        for line in header:
            if 'nmea utc' in line.lower():
                start_time = pd.to_datetime(re.split('= |\[', line)[1])
                hdr.update({'Start Time [UTC]': start_time.strftime('%Y-%m-%dT%H:%M:%S.000Z')})
            elif 'filename' in line.lower():
                hex_name = re.split('=', line)[1].strip()
                hdr.update({'Filename': hex_name})
            elif 'nmea latitude' in line.lower():
                start_lat = re.split('=', line)[1].strip()
                hdr.update({'Start Latitude [degrees]': start_lat})
            elif 'nmea longitude' in line.lower():
                start_lon = re.split('=', line)[1].strip()
                hdr.update({'Start Longitude [degrees]': start_lon})
            elif 'cruise id' in line.lower():
                cruise_id = re.split(':', line)[1].strip()
                hdr.update({'Cruise': cruise_id})
            else:
                pass

        self.header = hdr

    def parse_columns(self, columns):
        """
        Parse the column identifiers
        """

        column_dict = {}
        for line in columns:
            for i, x in enumerate(line.split()):
                try:
                    column_dict[i] = column_dict[i] + ' ' + x
                except:
                    column_dict.update({i: x})

        self.columns = column_dict

    def parse_data(self, data):
        """
        Parse the data in the contents, based on the column identifiers
        """

        # Create a dictionary to store the parsed data
        self.data = {x: [] for x in self.columns.keys()}

        # Parse the data into the dictionary based on the column location
        for line in data:
            if line.endswith('(avg)'):
                values = list(filter(None, re.split('  |\t| -', line)))
                for i, x in enumerate(values):
                    self.data[i].append(x)
            elif line.endswith('(sdev)'):
                values = list(filter(None, re.split('  |\t| -', line)))
                self.data[1].append(values[0])
            else:
                pass

        # Join the data and time for each measurement into a single item
        self.data[1] = [' '.join(item) for item in zip(self.data[1][::2], self.data[1][1::2])]

    def add_column_names(self):
        """
        With the parsed data and column identifiers, match up the data and column
        names based on the location
        """

        for key, value in self.columns.items():
            self.data[value] = self.data.pop(key)

    def load_content(self, filepath):
        """
        Load the content of a bottle (.btl) file

        Args:
            filepath - path to the bottle (.btl) file to be loaded
        Returns:
            self.content - a list of each line in the btl file, split at the newline
        """

        with open(filepath) as file:
            content = file.readlines()
        self.content = [x.strip() for x in content]

    def write_csv(self, savepath):
        """
        Write the parsed bottle information to a csv file
        """

        # First, put the data into a pandas dataframe
        df = pd.DataFrame.from_dict(self.data)

        # Now add the parsed header info into the dataframe
        for key, item in self.header.items():
            df[key] = item

        # Add in the cast number to the dataframe
        df = df.insert(0, "Cast", self.cast_number.zfill(3))

        # Write to a csv file
        df.to_csv(savepath + 'Cast' + str(self.cast_number) + '.sum', index=False)

    def parse_cast(self, filepath):
        """
        Function which integrates the separate cast functions to """

        # First, load the file content
        self.load_content(filepath)

        # Second, parse the file content
        header = []
        columns = []
        data = []
        for line in self.content:
            if line.startswith('*') or line.startswith('#'):
                header.append(line)
            else:
                try:
                    float(line[0])
                    data.append(line)
                except:
                    columns.append(line)

        # Third, parse the data, columns, and header info
        self.parse_columns(columns)
        self.parse_header(header)
        self.parse_data(data)

        # Add in the column names
        self.add_column_names()

---
### Process the ```.btl``` files

Get the name mapping from the SeaBird abbreviations to full names

In [4]:
sbe_name_map = pd.read_excel('../data/seabird_ctd_name_map.xlsx')
sbe_name_map.head()

,Short Name,Full Name,Friendly Name,Units,Notes/Comments
0,accM,Acceleration [m/s^2],acc M,m/s^2,NaN
1,accF,Acceleration [ft/s^2],acc F,ft/s^2,NaN
2,altM,Altimeter [m],alt M,m,NaN
3,altF,Altimeter [ft],alt F,ft,NaN
4,avgsvCM,"Average Sound Velocity [Chen-Millero, m/s]",avgsv-C M,"Chen-Millero, m/s",NaN


Navigate to the directory with the ```.btl``` files for the given cruise

In [7]:
BTLDIR = "/media/areed/Windows/Users/areed/Documents/OOI/Cruises/Papa-09/raw/SKQ202208S/"

In [9]:
for file in sorted(os.listdir(BTLDIR)):
    if file.endswith(".btl"):
        print(file)

SKQ202208S_001.btl
SKQ202208S_004.btl
SKQ202208S_005.btl
SKQ202208S_006.btl
SKQ202208S_007.btl
SKQ202208S_008.btl
SKQ202208S_009.btl
SKQ202208S_010.btl


Process the ```.btl``` files and append them with eachother into a dataframe

In [11]:
Bottles = pd.DataFrame()

for file in sorted(os.listdir(BTLDIR)):
    if file.endswith(".btl"):
        # Get the cast number from the file name
        cast_no = file[file.find(".")-3:file.find(".")]
        try:
            cast_no = int(cast_no)
        except ValueError:
            cast_no = cast_no.lstrip("0")
        
        # Initialize the CTD Cast object
        cast = Cast(cast_no)
        
        # Parse the cast data
        cast.parse_cast(BTLDIR+"/"+file)
        
        # Add in the cast number
        df = pd.DataFrame(cast.data)
        
        for key, item in cast.header.items():
            df.insert(1, key, item)

        df.insert(0, "Cast", cast.cast_number.zfill(3))
        
        # Save the results in a dataframe
        Bottles = Bottles.append(df, ignore_index=True)
        
Bottles.head()

/tmp/ipykernel_529382/3682776147.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Bottles = Bottles.append(df, ignore_index=True)
/tmp/ipykernel_529382/3682776147.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Bottles = Bottles.append(df, ignore_index=True)
/tmp/ipykernel_529382/3682776147.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Bottles = Bottles.append(df, ignore_index=True)
/tmp/ipykernel_529382/3682776147.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Bottles = Bottles.append(df, ignore_index=True)
/tmp/ipykernel_529382/3682776147.py:27: FutureWarning: The frame.append method is deprecated and will be rem

,Cast,Bottle Position,Start Time [UTC],Start Longitude [degrees],Start Latitude [degrees],Filename,Date Time,PrDM,DepSM,Latitude,...,C0S/m,C1S/m,Sal00,Sal11,Sbeox0V,Sbeox0ML/L,OxsolML/L,CStarAt0,CStarTr0,FlECO-AFL
0,001,1,2022-05-13T16:11:08.000Z,147 28.65 W,56 00.69 N,D:\CTD\SKQ202208S\SKQ202208S_001.hex,May 13 2022 16:51:58,1214.629,1200.071,56.00937,...,3.133915,3.133272,34.4474,34.4411,0.6145,0.4711,7.53789,0.0459,98.8579,0.0273 (avg)
1,004,1,2022-05-15T22:17:37.000Z,144 52.41 W,50 02.10 N,D:\CTD\SKQ202208S\SKQ202208S_004.hex,May 15 2022 23:39:33,4293.264,4213.913,50.03504,...,3.180953,3.180212,34.6765,34.6692,1.0535,3.0977,7.72138,0.0702,98.2607,0.0097 (avg)
2,004,2,2022-05-15T22:17:37.000Z,144 52.41 W,50 02.10 N,D:\CTD\SKQ202208S\SKQ202208S_004.hex,May 15 2022 23:39:42,4293.808,4214.441,50.03504,...,3.180975,3.180236,34.6765,34.6692,1.0535,3.0962,7.72137,0.0696,98.2753,-0.0007 (avg)
3,005,1,2022-05-17T20:40:34.000Z,144 55.35 W,50 06.85 N,D:\CTD\SKQ202208S\SKQ202208S_005.hex,May 17 2022 21:58:29,4175.501,4099.386,50.11418,...,3.175919,3.175175,34.6757,34.6684,1.0623,3.0960,7.72363,0.0594,98.5278,0.0059 (avg)
4,005,2,2022-05-17T20:40:34.000Z,144 55.35 W,50 06.85 N,D:\CTD\SKQ202208S\SKQ202208S_005.hex,May 17 2022 21:58:35,4175.909,4099.783,50.11418,...,3.175935,3.175193,34.6757,34.6685,1.0625,3.0993,7.72363,0.0593,98.5297,0.0013 (avg)


Reformat the Start Longitude and Start Latitude to be decimal degrees and Bottle Closure Time to "YYYY-mm-ddTHH:MM:SS.000Z"

In [12]:
def parse_start_latitude(x):
    try:
        if "S" in x:
            x = x.replace("N","").lstrip("0").strip().split()
            x = float(x[0]) + float(x[1])/60
            x = -1*x
        elif "N" in x:
            x = x.replace("N","").lstrip("0").strip().split()
            x = float(x[0]) + float(x[1])/60
        else:
            pass
    except:
        pass
    
    return x

def parse_start_longitude(x):
    try:
        if "W" in x:
            x = x.replace("N","").lstrip("0").strip().split()
            x = float(x[0]) + float(x[1])/60
            x = -1*x
        elif "E" in x:
            x = x.replace("N","").lstrip("0").strip().split()
            x = float(x[0]) + float(x[1])/60
        else:
            pass
    except:
        pass
    return x

def parse_start_time(x):
    try:
        x = pd.to_datetime(x).strftime("%Y-%m-%dT%H:%M:%S.000Z")
    except:
        pass
    return x

Bottles["Date Time"] = Bottles["Date Time"].apply(lambda x: pd.to_datetime(x).strftime("%Y-%m-%dT%H:%M:%S.000Z"))
Bottles["Start Time [UTC]"] = Bottles["Start Time [UTC]"].apply(lambda x:  parse_start_time(x))
Bottles["Start Latitude [degrees]"] = Bottles["Start Latitude [degrees]"].apply(lambda x: parse_start_latitude(x))
Bottles["Start Longitude [degrees]"] = Bottles["Start Longitude [degrees]"].apply(lambda x: parse_start_longitude(x))
Bottles["Filename"] = Bottles["Filename"].apply(lambda x: x.split("\\")[-1])

Rename the column title using the sbe_name_mapping

In [13]:
for colname in list(Bottles.columns.values):
    try:
        fullname = list(sbe_name_map[sbe_name_map['Short Name'].apply(lambda x: str(x).lower() == colname.lower()) == True]['Full Name'])[0]
        Bottles.rename({colname:fullname},axis='columns',inplace=True)
    except:
        pass

In [14]:
Bottles.head()

,Cast,Bottle Position,Start Time [UTC],Start Longitude [degrees],Start Latitude [degrees],Filename,Date Time,"Pressure, Digiquartz [db]","Depth [salt water, m]",Latitude [deg],...,Conductivity [S/m],"Conductivity, 2 [S/m]","Salinity, Practical [PSU]","Salinity, Practical, 2 [PSU]","Oxygen raw, SBE 43 [V]","Oxygen, SBE 43 [ml/l]","Oxygen Saturation, Garcia & Gordon [ml/l]","Beam Attenuation, WET Labs C-Star [1/m]","Beam Transmission, WET Labs C-Star [%]","Fluorescence, WET Labs ECO-AFL/FL [mg/m^3]"
0,001,1,2022-05-13T16:11:08.000Z,-147.4775,56.011500,SKQ202208S_001.hex,2022-05-13T16:51:58.000Z,1214.629,1200.071,56.00937,...,3.133915,3.133272,34.4474,34.4411,0.6145,0.4711,7.53789,0.0459,98.8579,0.0273 (avg)
1,004,1,2022-05-15T22:17:37.000Z,-144.8735,50.035000,SKQ202208S_004.hex,2022-05-15T23:39:33.000Z,4293.264,4213.913,50.03504,...,3.180953,3.180212,34.6765,34.6692,1.0535,3.0977,7.72138,0.0702,98.2607,0.0097 (avg)
2,004,2,2022-05-15T22:17:37.000Z,-144.8735,50.035000,SKQ202208S_004.hex,2022-05-15T23:39:42.000Z,4293.808,4214.441,50.03504,...,3.180975,3.180236,34.6765,34.6692,1.0535,3.0962,7.72137,0.0696,98.2753,-0.0007 (avg)
3,005,1,2022-05-17T20:40:34.000Z,-144.9225,50.114167,SKQ202208S_005.hex,2022-05-17T21:58:29.000Z,4175.501,4099.386,50.11418,...,3.175919,3.175175,34.6757,34.6684,1.0623,3.0960,7.72363,0.0594,98.5278,0.0059 (avg)
4,005,2,2022-05-17T20:40:34.000Z,-144.9225,50.114167,SKQ202208S_005.hex,2022-05-17T21:58:35.000Z,4175.909,4099.783,50.11418,...,3.175935,3.175193,34.6757,34.6685,1.0625,3.0993,7.72363,0.0593,98.5297,0.0013 (avg)


Add the Cruise ID/Name to the Bottle Data

In [16]:
Bottles.insert(0, "Cruise", "SKQ202208S")
Bottles.head()

,Cruise,Cast,Bottle Position,Start Time [UTC],Start Longitude [degrees],Start Latitude [degrees],Filename,Date Time,"Pressure, Digiquartz [db]","Depth [salt water, m]",...,Conductivity [S/m],"Conductivity, 2 [S/m]","Salinity, Practical [PSU]","Salinity, Practical, 2 [PSU]","Oxygen raw, SBE 43 [V]","Oxygen, SBE 43 [ml/l]","Oxygen Saturation, Garcia & Gordon [ml/l]","Beam Attenuation, WET Labs C-Star [1/m]","Beam Transmission, WET Labs C-Star [%]","Fluorescence, WET Labs ECO-AFL/FL [mg/m^3]"
0,SKQ202208S,001,1,2022-05-13T16:11:08.000Z,-147.4775,56.011500,SKQ202208S_001.hex,2022-05-13T16:51:58.000Z,1214.629,1200.071,...,3.133915,3.133272,34.4474,34.4411,0.6145,0.4711,7.53789,0.0459,98.8579,0.0273 (avg)
1,SKQ202208S,004,1,2022-05-15T22:17:37.000Z,-144.8735,50.035000,SKQ202208S_004.hex,2022-05-15T23:39:33.000Z,4293.264,4213.913,...,3.180953,3.180212,34.6765,34.6692,1.0535,3.0977,7.72138,0.0702,98.2607,0.0097 (avg)
2,SKQ202208S,004,2,2022-05-15T22:17:37.000Z,-144.8735,50.035000,SKQ202208S_004.hex,2022-05-15T23:39:42.000Z,4293.808,4214.441,...,3.180975,3.180236,34.6765,34.6692,1.0535,3.0962,7.72137,0.0696,98.2753,-0.0007 (avg)
3,SKQ202208S,005,1,2022-05-17T20:40:34.000Z,-144.9225,50.114167,SKQ202208S_005.hex,2022-05-17T21:58:29.000Z,4175.501,4099.386,...,3.175919,3.175175,34.6757,34.6684,1.0623,3.0960,7.72363,0.0594,98.5278,0.0059 (avg)
4,SKQ202208S,005,2,2022-05-17T20:40:34.000Z,-144.9225,50.114167,SKQ202208S_005.hex,2022-05-17T21:58:35.000Z,4175.909,4099.783,...,3.175935,3.175193,34.6757,34.6685,1.0625,3.0993,7.72363,0.0593,98.5297,0.0013 (avg)


Save the results

In [17]:
filename = "Papa-09_SKQ202208S_CTD_Bottle_Data_2022-11-16_ACR.xlsx"
filepath = "/home/areed/Documents/OOI/reedan88/QAQC_Sandbox/Ship_data/data/Papa-09/" + filename

In [18]:
Bottles.to_excel(filepath, index=False)